In [5]:
import subprocess
import glob
import os
from os.path import exists
import re
import pandas as pd
from multiprocessing import Pool

In [2]:
def process_csv_to_bed(input_csv, output_directory):
    # Ensure the output directory exists
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Load the CSV into a pandas DataFrame
    df = pd.read_csv(input_csv)

    # Derive the base filename without extension
    base_filename = os.path.splitext(os.path.basename(input_csv))[0]

    # Filter rows based on the conditions
    upregulated = df[(df['Fold'] > 1) & (df['FDR'] < 0.05)]
    downregulated = df[(df['Fold'] < -1) & (df['FDR'] < 0.05)]

    # Select the required columns and save to BED format
    upregulated_bed = os.path.join(output_directory, f"{base_filename}_Up.bed")
    downregulated_bed = os.path.join(output_directory, f"{base_filename}_Down.bed")

    upregulated[['seqnames', 'start', 'end', 'width', 'strand']].to_csv(upregulated_bed, sep='\t', header=False, index=False)
    downregulated[['seqnames', 'start', 'end', 'width', 'strand']].to_csv(downregulated_bed, sep='\t', header=False, index=False)
    print(f"Generated {upregulated_bed} and {downregulated_bed}\n")

# Execute the function for both CSV files
csv_directory = "/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/diffbind_results_DM1_DM2d_DM6_DM7"
DM1_DM2d_dir =  os.path.join(csv_directory,  "motif_DiffPeakAnno_DM1_DM2d")
DM6_DM7_dir =   os.path.join(csv_directory,  "motif_DiffPeakAnno_DM6_DM7")


process_csv_to_bed(os.path.join(csv_directory, "DiffPeakAnno_DM1_DM2d.csv"), DM1_DM2d_dir)
process_csv_to_bed(os.path.join(csv_directory, "DiffPeakAnno_DM6_DM7.csv"), DM6_DM7_dir)

Generated /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/diffbind_results_DM1_DM2d_DM6_DM7/motif_DiffPeakAnno_DM1_DM2d/DiffPeakAnno_DM1_DM2d_Up.bed and /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/diffbind_results_DM1_DM2d_DM6_DM7/motif_DiffPeakAnno_DM1_DM2d/DiffPeakAnno_DM1_DM2d_Down.bed

Generated /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/diffbind_results_DM1_DM2d_DM6_DM7/motif_DiffPeakAnno_DM6_DM7/DiffPeakAnno_DM6_DM7_Up.bed and /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/diffbind_results_DM1_DM2d_DM6_DM7/motif_DiffPeakAnno_DM6_DM7/DiffPeakAnno_DM6_DM7_Down.bed



### the bed file of peaks from Chipseeker

## compare enriched genes vs random background

In [3]:
import os
import subprocess
from multiprocessing import Pool

def find_motifs_task(bed_file):
    motif_directory = os.path.dirname(bed_file)
    output_folder = os.path.splitext(os.path.basename(bed_file))[0]
    output_path = os.path.join(motif_directory, output_folder)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Run the HOMER command
    cmd = [
        "findMotifsGenome.pl",
        bed_file,
        os.path.join('/media/HDD2/Genomes/Ath_Ensembl56', 'Arabidopsis_thaliana.TAIR10.dna.toplevel.fa'),
        output_path,
        "-p", "20",
        "-mset", "plants"
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)

    # Append both stderr and stdout to the log file
    with open(os.path.join(motif_directory, "HOMER.log"), "a") as log_file:
        log_file.write(result.stdout)
        log_file.write(result.stderr)

def run_find_motifs(directories):
    # Gather all bed files from the provided directories
    bed_files = []
    for directory in directories:
        bed_files.extend([os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.bed')])

    # Use Pool to run the tasks in parallel
    with Pool(4) as p:
        p.map(find_motifs_task, bed_files)



directories_to_search = [
   DM1_DM2d_dir,
   DM6_DM7_dir ]

run_find_motifs(directories_to_search)


## compare enriched up vs down regulated genes 

In [6]:
DM1_DM2d_beds = glob.glob( os.path.join(
    DM1_DM2d_dir,
    '*.bed'))
DM6_DM7_beds = glob.glob( os.path.join(
    DM6_DM7_dir,
    '*.bed'))

In [7]:
DM1_DM2d_beds

['/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/diffbind_results_DM1_DM2d_DM6_DM7/motif_DiffPeakAnno_DM1_DM2d/DiffPeakAnno_DM1_DM2d_Down.bed',
 '/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/diffbind_results_DM1_DM2d_DM6_DM7/motif_DiffPeakAnno_DM1_DM2d/DiffPeakAnno_DM1_DM2d_Up.bed']

In [8]:
DM6_DM7_beds

['/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/diffbind_results_DM1_DM2d_DM6_DM7/motif_DiffPeakAnno_DM6_DM7/DiffPeakAnno_DM6_DM7_Up.bed',
 '/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/diffbind_results_DM1_DM2d_DM6_DM7/motif_DiffPeakAnno_DM6_DM7/DiffPeakAnno_DM6_DM7_Down.bed']

In [9]:
def find_motifs_task_with_bg(bed_file, background_file, genome_path, species='plants'):
    motif_directory = os.path.dirname(bed_file)
    output_folder = os.path.splitext(os.path.basename(bed_file))[0] + '_vs_' + os.path.splitext(os.path.basename(background_file))[0]
    output_path = os.path.join(motif_directory, output_folder)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Run the HOMER command with background BED
    cmd = [
        "findMotifsGenome.pl",
        bed_file,
        genome_path,
        output_path,
        "-bg", background_file,
        "-p", "20",
        "-mset", species,
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)
    if result.stderr:
        print(result.stderr)

genome_fa = '/media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa'



In [12]:
# Run motif analysis for up-regulated regions using down-regulated as background
find_motifs_task_with_bg(
    os.path.join(DM1_DM2d_dir, 'DiffPeakAnno_' + 'DM1_DM2d' + '_Up.bed'),
    os.path.join(DM1_DM2d_dir, 'DiffPeakAnno_' + 'DM1_DM2d' + '_Down.bed'),
                     genome_fa)

# Run motif analysis for down-regulated regions using up-regulated as background
find_motifs_task_with_bg(
    os.path.join(DM1_DM2d_dir, 'DiffPeakAnno_' + 'DM1_DM2d' + '_Down.bed'),
    os.path.join(DM1_DM2d_dir, 'DiffPeakAnno_' + 'DM1_DM2d' + '_Up.bed'),
                 genome_fa)



	Position file = /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/diffbind_results_DM1_DM2d_DM6_DM7/motif_DiffPeakAnno_DM1_DM2d/DiffPeakAnno_DM1_DM2d_Up.bed
	Genome = /media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa
	Output Directory = /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/diffbind_results_DM1_DM2d_DM6_DM7/motif_DiffPeakAnno_DM1_DM2d/DiffPeakAnno_DM1_DM2d_Up_vs_DiffPeakAnno_DM1_DM2d_Down
	background position file: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/diffbind_results_DM1_DM2d_DM6_DM7/motif_DiffPeakAnno_DM1_DM2d/DiffPeakAnno_DM1_DM2d_Down.bed
	Using 20 CPUs
	Using Custom Genome
	Peak/BED file conversion summary:
		BED/Header formatted lines: 237
		peakfile formatted lines: 0

	Peak File Statistics:
		Total Peaks: 237
		Redundant Peak IDs: 29
		Peaks lacking information: 0 (need at least 5 columns per peak)
		Peaks with misformatted coordinates: 0 (should be integer)
		Peaks with misformatted strand: 0 (should be either +/- or 0/1)

	Redunant Peaks found: Remove o

In [13]:
#do the same for DM6_DM7
find_motifs_task_with_bg(os.path.join(DM6_DM7_dir, 'DiffPeakAnno_' + 'DM6_DM7' + '_Up.bed'),
                         os.path.join(DM6_DM7_dir, 'DiffPeakAnno_' + 'DM6_DM7' + '_Down.bed'),
                         genome_fa)
find_motifs_task_with_bg(os.path.join(DM6_DM7_dir, 'DiffPeakAnno_' + 'DM6_DM7' + '_Down.bed'),
                         os.path.join(DM6_DM7_dir, 'DiffPeakAnno_' + 'DM6_DM7' + '_Up.bed'),
                         genome_fa)



	Position file = /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/diffbind_results_DM1_DM2d_DM6_DM7/motif_DiffPeakAnno_DM6_DM7/DiffPeakAnno_DM6_DM7_Up.bed
	Genome = /media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa
	Output Directory = /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/diffbind_results_DM1_DM2d_DM6_DM7/motif_DiffPeakAnno_DM6_DM7/DiffPeakAnno_DM6_DM7_Up_vs_DiffPeakAnno_DM6_DM7_Down
	background position file: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/diffbind_results_DM1_DM2d_DM6_DM7/motif_DiffPeakAnno_DM6_DM7/DiffPeakAnno_DM6_DM7_Down.bed
	Using 20 CPUs
	Using Custom Genome
	Peak/BED file conversion summary:
		BED/Header formatted lines: 16
		peakfile formatted lines: 0

	Peak File Statistics:
		Total Peaks: 16
		Redundant Peak IDs: 0
		Peaks lacking information: 0 (need at least 5 columns per peak)
		Peaks with misformatted coordinates: 0 (should be integer)
		Peaks with misformatted strand: 0 (should be either +/- or 0/1)

	Peak file looks good!

	Peak/BED file co